In [ ]:
!pip install pyPPG==1.0.41
from pyPPG import PPG, Fiducials, Biomarkers
from pyPPG.datahandling import load_data
import pyPPG.preproc as PP
import pyPPG.fiducials as FP
import numpy as np
from google.colab import files
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Upload the file directly to Colab
uploaded = files.upload()
data_path = list(uploaded.keys())[0]  # Get the uploaded filename

start_sig = 0  # First sample of the signal to be analysed
end_sig = -1  # Last sample of the signal to be analysed
savingfolder = 'temp_dir'
savingformat = 'csv'

# Load the raw PPG signal
signal = load_data(data_path=data_path, start_sig=start_sig, end_sig=end_sig, use_tk=False)
signal.v = signal.v[0:20*signal.fs]  # 20-second long signal to be analysed

# Preprocess the PPG signal
signal.filtering = True
signal.fL = 0.5  # Lower cutoff frequency (Hz)
signal.fH = 12  # Upper cutoff frequency (Hz)
signal.order = 4  # Filter order
signal.sm_wins = {'ppg': 50, 'vpg': 10, 'apg': 10, 'jpg': 10}  # Smoothing windows

prep = PP.Preprocess(fL=signal.fL, fH=signal.fH, order=signal.order, sm_wins=signal.sm_wins)
signal.ppg, signal.vpg, signal.apg, signal.jpg = prep.get_signals(s=signal)

# Create a PPG class and extract fiducial points
s = PPG(signal)
fpex = FP.FpCollection(s=s)
fiducials = fpex.get_fiducials(s=s)
fp = Fiducials(fp=fiducials)

# Calculate Inter-Beat Interval (IBI)
peak_times = np.array(fiducials['sp']) / s.fs  # Fiducial points in seconds
ibi = np.diff(peak_times)  # Inter-Beat Interval in seconds

# Calculate heart rate (HR) in beats per minute (BPM)
hr = 60 / np.mean(ibi)
print(f'Calculated Heart Rate: {hr:.2f} BPM')

# Placeholder for ground truth heart rates (manually labeled data)
ground_truth_hr = [75, 77, 76, 74, 78]  # Example ground truth values for demonstration

# Truncate biomarkers to match the number of ground truth heart rates
num_samples = len(ground_truth_hr)
biomarkers = np.array([ibi[:num_samples]])  # Using IBI as the biomarker for demonstration

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(biomarkers.reshape(-1, 1), ground_truth_hr)

# Predicting heart rate
predicted_hr = rf.predict(biomarkers.reshape(-1, 1))

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(ground_truth_hr, predicted_hr)
print(f'Mean Absolute Error: {mae:.2f}')

Saving Sample_PPG_MAT_125Hz.mat to Sample_PPG_MAT_125Hz (12).mat
Calculated Heart Rate: 100.14 BPM
Mean Absolute Error: 0.53
